In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import SGD

np.random.seed(9074)

Using TensorFlow backend.


In [2]:
classes = 10

In [6]:
def mnist_data():
    
    img_rows = 28
    img_cols = 28
    
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = X_train.shape[1:]
    print("input_shape={}".format(input_shape))
    
    X_train = X_train.astype(np.float32) / 255
    X_test = X_test.astype(np.float32) / 255
    Y_train = np_utils.to_categorical(Y_train, classes)
    Y_test = np_utils.to_categorical(Y_test, classes)
    
    return (X_train, Y_train), (X_test, Y_test), input_shape

In [7]:
(X_train, Y_train), (X_test, Y_test), input_shape = mnist_data()

input_shape=(28, 28, 1)


In [8]:
def getModel(input_shape):
    filter_size = 32
    kernel_size = (3, 3)
    pool_size = (2, 2)
    
    model = Sequential()
    model.add(Conv2D(filter_size, kernel_size, padding='same',
                    input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filter_size, kernel_size, padding='same'))
    model.add(MaxPooling2D(pool_size))
    model.add(Dropout(0,2))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                 metrics=['accuracy'])
    
    return model

In [9]:
model = getModel(input_shape = input_shape)

In [15]:
model.fit(X_train, Y_train, epochs=1, batch_size=32, verbose=1,validation_split=0.25)

Train on 45000 samples, validate on 15000 samples
Epoch 1/1
45000/45000 [==============================] - 182s 4ms/step - loss: 0.1488 - acc: 0.9540 - val_loss: 0.0789 - val_acc: 0.9763


In [22]:
eva=model.evaluate(X_test, Y_test, verbose=1)
print('Test loss: ', rst[0], ' accuracy: ', rst[1])

rst = model.predict(X_test[3:4])
print(rst)

AttributeError: 'KerasClassifier' object has no attribute 'evaluate'

## grid search

In [23]:
def getModel(input_shape, dropout_rate='0.25'):
    filter_size = 32
    kernel_size = (3, 3)
    pool_size = (2, 2)
    
    model = Sequential()
    model.add(Conv2D(filter_size, kernel_size, padding='same',
                    input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filter_size, kernel_size, padding='same'))
    model.add(MaxPooling2D(pool_size))
    model.add(Dropout(0,2))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                 metrics=['accuracy'])
    
    return model

In [24]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=getModel, epochs=2, batch_size=32,
                       verbose=1, input_shape=input_shape)


In [25]:
from sklearn.model_selection import GridSearchCV

dropout_rate = [0.1, 0.2, 0.25, 0.5]
param_grid = dict(dropout_rate = dropout_rate)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train, Y_train)


Epoch 1/2
48000/48000 [==============================] - 163s 3ms/step - loss: 0.1408 - acc: 0.9569
Epoch 2/2
48000/48000 [==============================] - 46s 962us/step
Epoch 1/2
 2208/48000 [>.............................] - ETA: 2:40 - loss: 0.6933 - acc: 0.7785

KeyboardInterrupt: 

# checkpoint

In [26]:
from keras.callbacks import ModelCheckpoint

filepath = 'weights-improvement-{epoch:02d}-{val_acc: 2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                            save_best_only=True, mode='max')

In [28]:
model_history=model.fit(X_train, Y_train, epochs=1, batch_size=32, verbose=1,
                       validation_split=0.25, callbacks=[checkpoint])

Train on 45000 samples, validate on 15000 samples
Epoch 1/1
45000/45000 [==============================] - 177s 4ms/step - loss: 0.1571 - acc: 0.9520 - val_loss: 0.0681 - val_acc: 0.9813

Epoch 00001: val_acc improved from -inf to 0.98127, saving model to weights-improvement-01- 0.981267.hdf5


In [30]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=getModel, epochs=100, batch_size=32,
                       verbose=1, input_shape=input_shape)

In [31]:
model.load_weights("weights.best.hdf5")

AttributeError: 'KerasClassifier' object has no attribute 'load_weights'

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor)